# FEUP - AC
## Banking - Predicting a Loan Outcome

### Business understanding

##### Analysis of requirements with the end user
Text

##### Definition of business goals
Text

##### Translation of business goals into data mining goals
Text

### Exploratory data analysis

Let us start by importing the datasets.

In [56]:
import pandas as pd
import numpy as np

# Global settings
pd.set_option('display.max_columns', None)

COMP_DATA_SOURCE = "comp_data/"
DEV_DATA_SOURCE = "dev_data/"

# Read in the data; merge data for tables other than loans since they do not influence merges
accounts = pd.read_csv(DEV_DATA_SOURCE + 'account.csv', sep=';')

cards_dev = pd.read_csv(DEV_DATA_SOURCE + 'card.csv', sep=';')
cards_comp = pd.read_csv(COMP_DATA_SOURCE + 'card.csv', sep=';')
cards = pd.concat([cards_dev, cards_comp])

clients = pd.read_csv(DEV_DATA_SOURCE + 'client.csv', sep=';')

dispositions = pd.read_csv(DEV_DATA_SOURCE + 'disp.csv', sep=';')

districts = pd.read_csv(DEV_DATA_SOURCE + 'district.csv', sep=';')

transactions_dev = pd.read_csv(DEV_DATA_SOURCE + 'trans.csv', sep=';')
transactions_comp = pd.read_csv(COMP_DATA_SOURCE + 'trans.csv', sep=';')
transactions = pd.concat([transactions_dev, transactions_comp])

loans_dev = pd.read_csv(DEV_DATA_SOURCE + 'loan.csv', sep=';')
loans_comp = pd.read_csv(COMP_DATA_SOURCE + 'loan.csv', sep=';')

loans_dev.head()

/tmp/ipykernel_11759/482001259.py:23: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions_dev = pd.read_csv(DEV_DATA_SOURCE + 'trans.csv', sep=';')


,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033,-1
1,5316,1801,930711,165960,36,4610,1
2,6863,9188,930728,127080,60,2118,1
3,5325,1843,930803,105804,36,2939,1
4,7240,11013,930906,274740,60,4579,1


Plenty of analysis with graphs, distributions, outliers...

### Data preprocessing

Text

#### Processing accounts

The account spreadsheet relates it with the district and date in which it was created, as well with the frequency of issuance of statements with the owner.

The date comes in the from YYMMDD, which is easily extractable to the three different columns that it is aggregating. This will aid the model in detecting important patterns in the year of creation, for example, indicating a global economic crisis or prosperity when the person has created the account.

In [57]:
import src.preprocess.accounts as accpp
accounts = accpp.preprocess_accounts(accounts)
accounts.head()

,account_id,district_id,account_frequency,account_year,account_month,account_day
0,576,55,monthly issuance,93,1,1
1,3818,74,monthly issuance,93,1,1
2,704,55,monthly issuance,93,1,1
3,2378,16,monthly issuance,93,1,1
4,2632,24,monthly issuance,93,1,2


#### Processing credit cards

In the credit cards relation, the same concept applies to the date of issuance.

MORE TEXT

In [58]:
from src.preprocess.cards import preprocess_cards

cards = preprocess_cards(cards)
cards.head()

,card_id,disp_id,type,year,month,day
0,1005,9285,classic,93,11,7
1,104,588,classic,94,1,19
2,747,4915,classic,94,2,5
3,70,439,classic,94,2,8
4,577,3687,classic,94,2,15


#### Processing clients

In the clients relation, two foreign references and a date appear once again. However, regarding the latter one, there is an important difference: the day is the actual day plus fifty, if the client is a female, and so we can extract the gender too, here.

In [59]:
from src.preprocess.clients import preprocess_clients

clients = preprocess_clients(clients)
clients.head()

,client_id,district_id,year,month,day,gender
0,1,18,70,12,13,female
1,2,1,45,2,4,male
2,3,1,40,10,9,female
3,4,5,56,12,1,male
4,5,5,60,7,3,female


#### Processing dispositions

The disposition describes the rights of clients to operate accounts: only "owners" can ask for loans and issue permanent orders.

There is no clear way to merge this table with loans later on, since the loans table does not refer the disposition used for the loan. Instead, let's focus on the number of dispositions and the ratio of owners of an account, which will give an impression of the nature of the people able to control the account. All humans fight and cheat on each other, or build trust together. Let's discover that pattern in Czech Republic.

In [60]:
from src.preprocess.dispositions import preprocess_dispositions
dispositions = preprocess_dispositions(dispositions)
dispositions.head()

,disp_id,client_id,account_id,type,number_account_dispositions
0,1,1,1,OWNER,1
1,2,2,2,OWNER,2
2,3,3,2,DISPONENT,2
3,4,4,3,OWNER,2
4,5,5,3,DISPONENT,2


#### Processing districts

The districts relation is quite dense, describing the demographic data of a region. We can extract interesting measures here:

- The commited crimes growth ratio, derived from the evolution of the number of crimes from '95 to '96
- The unemployment rate growth ratio, derived from the evolution of the employment rate from '95 to '96

We also acknowledged that there was one region where the number of crimes and unemployment rate in '95 was not known (`?` was the appearance in the respective column). In order not to discard the entire region data, we'll simply assume that number to be the same as the matching record in '96.

In [61]:
from src.preprocess.districts import preprocess_districts
districts = preprocess_districts(districts)
districts.head()

,district_id,district_no_inhabitants,district_no_cities,district_urban_inhabitants_ratio,district_average_salary,district_unemployment_rate,districts_entrepreneurs_ratio,district_crimes_per_inhabitant,district_crime_growth,district_unemploymant_growth
0,1,1204953,1,100.0,12541,0.43,0.167,0.082250,1.156752,1.482759
1,2,88884,5,46.7,8507,1.85,0.132,0.030084,1.238536,1.107784
2,3,75232,5,41.7,8980,2.21,0.111,0.037391,0.996105,1.133333
3,4,149893,6,67.4,9753,5.05,0.109,0.039308,1.123570,1.088362
4,5,95616,6,51.4,9307,4.43,0.118,0.031794,1.162080,1.150649


#### Processing loans

The loans relation references an account and the date of loan, which can be extracted. The duration seems to be in months, so let us improve the name of the column. The amounts seems to be in `Czech korunas`, the czech currency, as hinted by the list of regions; we'll convert it to euros for better visualization and understanding of the upcoming results.

In [62]:
from src.preprocess.loans import preprocess_loans
loans_comp = preprocess_loans(loans_comp)
loans_dev = preprocess_loans(loans_dev)
loans_dev.head()

,loan_id,account_id,amount,duration_months,monthly_payment,status,year,month,day
0,5314,1787,3950.66,12,329.22,-1,93,7,5
1,5316,1801,6801.64,36,188.93,1,93,7,11
2,6863,9188,5208.20,60,86.80,1,93,7,28
3,5325,1843,4336.23,36,120.45,1,93,8,3
4,7240,11013,11259.84,60,187.66,1,93,9,6


#### Processing transactions

This table contains a lot of information that we cannot directly relate to a loan.

Let us first extract the date and make the withdrawal amounts actually negative, to give the model an impression of the direction of the money flow in and out of the bank.

For now, let us extract some statistical data related to the accounts, which we'll relate to the loans table.

In [63]:
from src.preprocess.transactions import preprocess_transactions
transactions = preprocess_transactions(transactions)
transactions.head()

,trans_id,account_id,type,operation,amount,balance,k_symbol,bank,account,year,month,day
0,1548749,5270,credit,credit in cash,32.79,32.79,NaN,NaN,NaN,93,1,13
1,1548750,5270,credit,collection from another bank,1833.98,1866.76,NaN,IJ,80269753.0,93,1,14
2,3393738,11265,credit,credit in cash,40.98,40.98,NaN,NaN,NaN,93,1,14
3,3122924,10364,credit,credit in cash,45.08,45.08,NaN,NaN,NaN,93,1,17
4,1121963,3834,credit,credit in cash,28.69,28.69,NaN,NaN,NaN,93,1,19


#### Feature engineering

Text

##### Getting the average account balance based on the transactions

In [64]:
from src.feature_engineering.merge import merge_account_transactions
accounts = merge_account_transactions(accounts, transactions)
accounts.head()

,account_id,district_id,account_frequency,account_year,account_month,account_day,account_average_balance
0,576,55,monthly issuance,93,1,1,995.213406
1,3818,74,monthly issuance,93,1,1,1755.200383
2,704,55,monthly issuance,93,1,1,1424.108906
3,2378,16,monthly issuance,93,1,1,2376.927362
4,2632,24,monthly issuance,93,1,2,1326.053913


##### Clusters

In [65]:
import src.stats.clusters as clusters
#clusters.cluster_monthly_payment_per_loan_amount(loans_dev)
#clusters.cluster_monthly_payment_per_loan_amount(loans_comp)

In [66]:
table = clusters.cluster_transactions(transactions)
accounts = pd.merge(accounts, table, on='account_id', how='left')
accounts.head()

(3724, 10)


,account_id,district_id,account_frequency,account_year,account_month,account_day,account_average_balance,transaction_profile
0,576,55,monthly issuance,93,1,1,995.213406,2
1,3818,74,monthly issuance,93,1,1,1755.200383,3
2,704,55,monthly issuance,93,1,1,1424.108906,3
3,2378,16,monthly issuance,93,1,1,2376.927362,2
4,2632,24,monthly issuance,93,1,2,1326.053913,3


In [67]:
new_districts_table = clusters.cluster_districts(districts)
print(districts.shape)
print(new_districts_table.shape)

new_districts_table.head()

(77, 11)
(77, 3)


,district_id,districts_profile,district_average_salary
0,1,1,12541
1,2,0,8507
2,3,8,8980
3,4,2,9753
4,5,0,9307


#### Merging the data

The model will be working on the `loans` table, since the target variable belongs to it. The one foreign key of that table is `account_id`; so, to feed the model with more information about the context of the loan, we should relate concepts to the account for each table.

##### Relating clients to districts

In [68]:
import src.feature_engineering.merge as merge

clients = merge.merge_client_districts(clients, new_districts_table)
clients.rename(columns={'district_id': 'client_district_id'}, inplace=True)
clients.head()

,client_id,year,month,day,gender,owner_districts_profile,owner_district_average_salary
0,1,70,12,13,female,8,8968
1,2,45,2,4,male,1,12541
2,3,40,10,9,female,1,12541
3,4,56,12,1,male,0,9307
4,5,60,7,3,female,0,9307


##### Relating dispositions to clients

In [69]:
dispositions = merge.merge_dispositions_clients(dispositions, clients)
dispositions.head()

,disp_id,account_id,type,number_account_dispositions,client_year,client_month,client_day,client_gender,owner_districts_profile,owner_district_average_salary
0,1,1,OWNER,1,70,12,13,female,8,8968
1,2,2,OWNER,2,45,2,4,male,1,12541
2,3,2,DISPONENT,2,40,10,9,female,1,12541
3,4,3,OWNER,2,56,12,1,male,0,9307
4,5,3,DISPONENT,2,60,7,3,female,0,9307


##### Relating dispositions to cards

In [70]:
dispositions = merge.merge_dispositions_cards(dispositions, cards) # cards are not enough; too many nans, check and discard
dispositions.head()


,disp_id,account_id,type,number_account_dispositions,client_year,client_month,client_day,client_gender,owner_districts_profile,owner_district_average_salary,card_type
0,1,1,OWNER,1,70,12,13,female,8,8968,missing
1,2,2,OWNER,2,45,2,4,male,1,12541,missing
2,3,2,DISPONENT,2,40,10,9,female,1,12541,missing
3,4,3,OWNER,2,56,12,1,male,0,9307,missing
4,5,3,DISPONENT,2,60,7,3,female,0,9307,missing


##### Relating owner dispositions to accounts

Only owners can ask for loans. This way, we can directly relate an account to its owner client.

In [71]:
accounts = merge.merge_account_dispositions(accounts, dispositions)
accounts.head()

,account_id,district_id,account_frequency,account_year,account_month,account_day,account_average_balance,transaction_profile,number_account_dispositions,owner_year,owner_month,owner_day,owner_gender,owner_districts_profile,owner_district_average_salary,card_type
0,576,55,monthly issuance,93,1,1,995.213406,2,2,36,1,11,female,3,10673,missing
1,3818,74,monthly issuance,93,1,1,1755.200383,3,2,35,4,2,male,1,12541,missing
2,704,55,monthly issuance,93,1,1,1424.108906,3,2,45,1,14,male,4,8620,missing
3,2378,16,monthly issuance,93,1,1,2376.927362,2,1,75,3,24,female,0,8427,missing
4,2632,24,monthly issuance,93,1,2,1326.053913,3,1,38,8,12,male,5,8991,missing


In [72]:
accounts = merge.merge_accounts_districts(accounts, new_districts_table)
accounts.rename(columns={'district_id': 'account_district_id'}, inplace=True)
accounts.head()

,account_id,account_frequency,account_year,account_month,account_day,account_average_balance,transaction_profile,number_account_dispositions,owner_year,owner_month,owner_day,owner_gender,owner_districts_profile,owner_district_average_salary,card_type,account_districts_profile,account_district_average_salary
0,576,monthly issuance,93,1,1,995.213406,2,2,36,1,11,female,3,10673,missing,2,8743
1,3818,monthly issuance,93,1,1,1755.200383,3,2,35,4,2,male,1,12541,missing,3,10673
2,704,monthly issuance,93,1,1,1424.108906,3,2,45,1,14,male,4,8620,missing,2,8743
3,2378,monthly issuance,93,1,1,2376.927362,2,1,75,3,24,female,0,8427,missing,0,8427
4,2632,monthly issuance,93,1,2,1326.053913,3,1,38,8,12,male,5,8991,missing,5,8991


##### Loading the loans table with data

In [73]:
accounts.rename(columns = {'year':'account_year', 'month':'account_month', 'day':'account_day', 'frequency': 'account_frequency', 'district_id': 'account_district_id'}, inplace=True)
loans_dev = loans_dev.merge(accounts, on="account_id", how="left")
loans_comp = loans_comp.merge(accounts, on="account_id", how="left")
loans_dev.drop(columns=['account_id'], inplace=True)
loans_comp.drop(columns=['account_id'], inplace=True)
loans_dev.head()

,loan_id,amount,duration_months,monthly_payment,status,year,month,day,account_frequency,account_year,account_month,account_day,account_average_balance,transaction_profile,number_account_dispositions,owner_year,owner_month,owner_day,owner_gender,owner_districts_profile,owner_district_average_salary,card_type,account_districts_profile,account_district_average_salary
0,5314,3950.66,12,329.22,-1,93,7,5,weekly issuance,93,3,22,502.047500,4,1,47,7,22,female,0,9650,missing,0,9650
1,5316,6801.64,36,188.93,1,93,7,11,monthly issuance,93,2,13,2134.584324,4,1,68,7,22,male,9,8369,missing,9,8369
2,6863,5208.20,60,86.80,1,93,7,28,monthly issuance,93,2,8,1232.007083,4,1,36,6,2,male,8,8390,missing,8,8390
3,5325,4336.23,36,120.45,1,93,8,3,monthly issuance,93,1,30,1692.519600,4,1,40,4,20,female,2,10045,missing,9,8754
4,7240,11259.84,60,187.66,1,93,9,6,weekly issuance,93,2,14,2343.778889,4,1,78,9,7,male,0,8288,missing,1,12541


##### Feature selection and engineering on the loans table

In [74]:
import src.preprocess.util as utils
loans_dev['age_at_loan'] = loans_dev.apply(lambda row: utils.extract_age(row), axis=1)
loans_comp['age_at_loan'] = loans_comp.apply(lambda row: utils.extract_age(row), axis=1)

loans_dev = loans_dev.drop(columns=["account_month", "account_day", "month", "day", "owner_day", "owner_month", "owner_gender", "account_year",  "owner_year", "year"])
loans_comp = loans_comp.drop(columns=["account_month", "account_day", "month", "day", "owner_day", "owner_month", "owner_gender", "account_year",  "owner_year", "year"])

loans_dev.head(5)

,loan_id,amount,duration_months,monthly_payment,status,account_frequency,account_average_balance,transaction_profile,number_account_dispositions,owner_districts_profile,owner_district_average_salary,card_type,account_districts_profile,account_district_average_salary,age_at_loan
0,5314,3950.66,12,329.22,-1,weekly issuance,502.047500,4,1,0,9650,missing,0,9650,45
1,5316,6801.64,36,188.93,1,monthly issuance,2134.584324,4,1,9,8369,missing,9,8369,24
2,6863,5208.20,60,86.80,1,monthly issuance,1232.007083,4,1,8,8390,missing,8,8390,57
3,5325,4336.23,36,120.45,1,monthly issuance,1692.519600,4,1,2,10045,missing,9,8754,53
4,7240,11259.84,60,187.66,1,weekly issuance,2343.778889,4,1,0,8288,missing,1,12541,14


### Training the model

#### One hot encoding

In [75]:
print(len(loans_dev.columns) == len(loans_comp.columns))

loans_dev = utils.extract_categorical(loans_dev, "account_frequency")
loans_comp = utils.extract_categorical(loans_comp, "account_frequency")
loans_dev = utils.extract_categorical(loans_dev, "card_type")
loans_comp = utils.extract_categorical(loans_comp, "card_type")
#loans_dev = utils.extract_categorical(loans_dev, "owner_gender")
#loans_comp = utils.extract_categorical(loans_comp, "owner_gender")

loans_dev.head()

True


KeyError: 'card_type'

#### Splitting the data for training

Text

In [ ]:
from src.model.model import train_test_split_unbalanced
print(loans_dev.shape)
# experiment strategies here and inside the balancing function
x_train, x_test, y_train, y_test = train_test_split_unbalanced(loans_dev, "status", sampling_strategy="smote_enn", sort_by_date=False, train_ratio=0.8, random_state=0)
print(x_train.shape)

(328, 16)
(284, 15)


#### Predicting and evaluating

Evaluate classification model performance with loans table

code the categorical feature into multiple binary features to allow the model to work!! (TBD)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, roc_auc_score
from sklearn.svm import LinearSVC
from src.model.model import self_confidence_score
from sklearn.pipeline import Pipeline

# Create a Random Forest Model after selecting best features
#clf = RandomForestClassifier(n_estimators=200, random_state=0)
#experiment with other models here...
clf = Pipeline([
  #('feature_selection', SelectFromModel(LinearSVC(C=0.01, penalty="l1", dual=False))), # not improving that much
  ('classification', RandomForestClassifier(n_estimators=1000, random_state=0, warm_start=True))
])

# Train it with data
clf.fit(x_train, y_train)

if len(y_test) > 0: # not training with entire dataset
    # Predict the test data for dev and evaluate
    predictions_dev = clf.predict(x_test)
    predictions_dev_prob = clf.predict_proba(x_test)

    # Evaluate
    print('Recall: ', recall_score(y_test, predictions_dev, pos_label=-1))
    print('Accuracy: ', accuracy_score(y_test, predictions_dev))
    print('Precision: ', precision_score(y_test, predictions_dev, pos_label=-1))
    print('F-Score: ', f1_score(y_test, predictions_dev, pos_label=-1))
    print("AUROC: ", roc_auc_score(y_test, predictions_dev_prob[:, 1]))
    print("Dev self_confidence_score: ", self_confidence_score(predictions_dev_prob[:, 1]))

    # Write the dev predictions to a file
    predictions_dev_id = x_test["loan_id"]
    predictions_dev_debt = [x[0] for x in predictions_dev_prob]
    predictions_df_dev = pd.DataFrame({"Id": predictions_dev_id, "Predicted": predictions_dev_debt, "Actual": y_test})
    predictions_df_dev.to_csv("output/predictions_dev.csv", index=False)

# Predict the competition data with probabilities
loans_comp_indep = loans_comp.drop(axis=1, columns=["status"])
predictions_comp_prob = clf.predict_proba(loans_comp_indep)
predictions_comp = clf.predict(loans_comp_indep)
print("Competition self_confidence_score: ", self_confidence_score(predictions_comp_prob[:, 1]))

# Write the comp predictions to a file
predictions_comp_debt = [x[0] for x in predictions_comp_prob]
predictions_comp_id = loans_comp["loan_id"].tolist()
predictions_df_comp = pd.DataFrame({"Id": predictions_comp_id, "Predicted": predictions_comp_debt})
predictions_df_comp.to_csv("output/predictions_comp.csv", index=False)

# Output statistics and compare to other models

Recall:  0.8333333333333334
Accuracy:  0.8333333333333334
Precision:  0.3333333333333333
F-Score:  0.47619047619047616
AUROC:  0.9013888888888889
Dev self_confidence_score:  0.49348484848484847
Competition self_confidence_score:  0.41201694915254267
